In [ ]:
import os
import csv
import copy
import fnmatch
import numpy as np
import collections
import pandas as pd
from subprocess import call
from itertools import repeat
from itertools import dropwhile
from IPython.display import display
from pandas import read_csv, DataFrame, Index, MultiIndex

# Set up Environment

In [ ]:
path = '/autofs/space/lilli_001/users'
NARC = os.path.join('%s/DARPA-ARC_RER' % path)
NARC_analysis = os.path.join('%s/DARPA-Scripts/tutorials/darpa_pipelines_EH/darpa_newARC_scripts/params' % path)

scan_list=[]
subjs_list=[]

unpack_log='unpack.log'
raw_cfg= 'cfg_raw.txt'
new_cfg='cfg.txt'

# Generate Subject and Scan lists

##### Import *SPECIFIC* scans from BOURGET | FIRST, manually modify subjlist.txt, scan.txt

In [ ]:
def get_scans_subjects(analysis_path,scan_list,subjs_list):
    with open('%s/subjlist.txt' % analysis_path ,'r') as sublist, \
    open('%s/scan.txt' % analysis_path ,'r') as scanlist:
        for subject,scan in zip(sublist,scanlist):
            subjs_list.append(subject.strip())
            scan_list.append(scan.strip())
    return

get_scans_subjects(NARC_analysis,scan_list,subjs_list)

# unpackdcmsdir P.1

In [ ]:
def unpacksdcmdir_step1(analysis_path,scan_list,subjs_list,raw_data_path):
    with open('%s/unpacksdcmdir_step1.csh' % analysis_path ,'w') as unpacksdcmdir_1:
        for scan,subject in zip(scan_list,subjs_list):
            unpacksdcmdir_1.write('unpacksdcmdir -src %s -targ %s/%s/RAW -scanonly %s/%s/RAW/%s;\n' \
            % (scan, raw_data_path, subject, raw_data_path, subject, raw_cfg))
        print "Next: manually source unpacksdcmdir_step1.csh"
    return

unpacksdcmdir_step1(NARC_analysis,scan_list,subjs_list,NARC)

# Create Configuration File

## Extract cfg data from unpack.log
##### NOTE: unpack.log is used in lieu of cfg_raw.txt (from the first unpackdcmsdir step) to generate cfg.txt 

In [ ]:
cfg_data={'a':[],'b':[],'c':[],'d':[]}

def extract_unpack_data(raw_data_path,subjs_list,unpack_log,cfg_data): 
    for subject in subjs_list:
        infile=os.path.join(raw_data_path,subject,'RAW',unpack_log)
        with open(infile,'r') as in_txt:
            for line in in_txt:
                if 'ok' in line:
                    c=line.strip().split()
                    n=str(c[0])
                    scan=str(c[1])
                    num=str(c[7])
                    cfg_data['a'].append(n)
                    dicom='DICOM'
                    cfg_data['b'].append(scan)
                    cfg_data['c'].append(dicom)
                    cfg_data['d'].append(num)
    return

extract_unpack_data(NARC,subjs_list,unpack_log,cfg_data)

## Save raw cfg and generate new cfg

In [ ]:
def format_cfg(raw_data_path,subjs_list,raw_cfg,new_cfg,cfg_data):
    for subject in subjs_list:
        orig_cfg_subdir = os.path.join(raw_data_path,"%s/RAW/cfg.txt" % subject.rstrip(' \n'))
        with open(orig_cfg_subdir, 'w') as form_cfg:
            for a,b,c,d in zip(cfg_data['a'],cfg_data['b'],cfg_data['c'],cfg_data['d']):
                form_cfg.write('\t'.join([a,b,c,d]) + '\n')
    return

format_cfg(NARC,subjs_list,raw_cfg,new_cfg,cfg_data)

# unpackdcmsdir P.2

In [ ]:
def unpacksdcmdir_step1(analysis_path,scan_list,subjs_list,raw_data_path): 
    with open('%s/unpacksdcmdir_step2.csh' % analysis_path ,'w') as unpacksdcmdir_2, \
    open('%s/subjlist.txt' % analysis_path ,'r') as sublist:
        for scan,subject in zip(scan_list,subjs_list):    
            subdir = os.path.join(raw_data_path,"%s/RAW/cfg.txt" % subject.rstrip(' \n'))
            unpacksdcmdir_2.write('unpacksdcmdir -src %s -targ %s/%s/RAW -fsfast -cfg %s; \n'\
            % (scan.rstrip(' \n'), raw_data_path.rstrip(' \n'), subject.rstrip(' \n'), subdir))
        print "Next: source unpacksdcmdir_step2.csh"
    return

unpacksdcmdir_step1(NARC_analysis,scan_list,subjs_list,NARC)

# mri_convert

In [ ]:
dcm2nii='mri_dcm2nii_converter.csh'

path = '/autofs/space/lilli_001/users'
NARC = os.path.join('%s/DARPA-ARC_RER' % path)
NARC_analysis = os.path.join('%s/DARPA-Scripts/tutorials/darpa_pipelines_EH/darpa_newARC_scripts/params' % path)

def mri_dcm2nii(analysis_path, mri_converter, subjs_list,cfg_data):
    with open('%s/%s' % (analysis_path, mri_converter),'w') as scanlist:
        for subject in subjs_list:
            for scan,dcm in zip(cfg_data['b'],cfg_data['d']):   
                cwd=os.path.join(NARC,subject,'RAW',scan)
                for root, dirs, files in os.walk(cwd,topdown=False):
                    if dcm in os.listdir(root):
                        s=os.path.join(root,dcm)
                        scanlist.write('mri_convert %s %s/%s.nii; \n' % (s,root,scan))
    return

mri_dcm2nii(NARC_analysis, dcm2nii, subjs_list,cfg_data)

# Additional Functions (OPTIONAL)

### Read Configuration File

In [ ]:
# def read_cfg(path,subjid,doc): 
#     insert=os.path.join(path, subjid, doc)
#     doc=open(insert,'r' )
#     print doc.read()
    
# read_cfg('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH/','020043MR01','unpack.log')

### Loop to search for first DICOM in series (if not known from cfg file)

In [ ]:
# for subject in subjs_list:
#     bold_path= '%s/mri/%s/%s' % (NARC, subject, newARC_BOLD)
#     d1dir= os.listdir(bold_path)
#     for subdir in d1dir:
#         if subdir.startswith('0'):
#             subzir=os.path.join(bold_path,subdir)
#             for root, dirs, files in os.walk(os.path.join(bold_path,subdir)):
#                 for filename in fnmatch.filter(files, "*ARC_BOLD_RER*"):
#                     zir=sorted(os.listdir(os.path.join(bold_path,subdir)))
#                     if zir[0].startswith('MR'):
#                         inp=os.path.join(bold_path,subdir,zir[0])
#                         zirz.append(inp)
#                     elif zir[1].startswith('MR'):
#                         inp=os.path.join(bold_path,subdir,zir[1])
#                         zirz.append(inp)
                        
# with open('%s/%s' % (NARC_analysis, mri_converter),'w') as scanlist:
# #     del zirz[55:58]
#     for z,subject in zip(zirz,subjs_list):
#         print z
#         print subject
#         subject=subject.lower()[:-3]
#         scanlist.write('mri_convert %s %s/%s/msit_bsm/func/msit.%s.func.nii \n' % (z,indiv_pth,subject,subject))
        

### Import *ALL* scans from BOURGET
##### i.e. Automatically generate FINDSESS.txt, subjlist.txt, scan.txt

In [ ]:
#----------------------------#
# Generate Step 1,2 C shells
#----------------------------#
# os.system('cd %s; touch subjlist.txt scan.txt unpacksdcmdir_step1.csh unpacksdcmdir_step2.csh' % NARC_analysis)

#os.system('findsession -p res > %s/FINDSESS.txt' % NARC_analysis)

# def get_scans_subjects(path):
#     with open('%s/FINDSESS.txt' % path ,'r') as in_txt, \
#         open('%s/subjlist.txt' % path ,'w') as sublist, \
#         open('%s/scan.txt' % path ,'w') as scanlist:
#         for line in in_txt:
#                 if 'SUBJECT' in line:
#                     sub=line.strip('SUBJECT:  ')
#                     sublist.write('%s \n' % sub.strip())
#                     subject_list.append(sub)
#                 elif 'PATH' in line:
#                     scan=line.strip('PATH   :')
#                     scanlist.write('%s \n' % scan.strip())
#                     transfer_scanlist.append(sub)
#     return
#
# get_scans_subjects(NARC)

### Fix this cell to source FreeSurfer6 and enable C shell scripting in Jupyter

In [ ]:
# from subprocess import call
# import os

# call(['cd %s' % NARC_analysis],shell=True) 
# call(['source /usr/local/freesurfer/nmr-stable60-env'],shell=True)
# call(['source %s/unpacksdcmdir_step1.csh' % NARC_analysis],shell=True)

### DICOM to NIFTI - ARCHIVE

In [ ]:
# scan_list=['MEMPRAGE','BOLD']

# path=[]
# output=[]

# def dicom2nii_get_path(path):
#     for subject in subject_list:
#         if subject in os.listdir(path):
#             for scan in scan_list:
#                 if scan in os.listdir(os.path.join(path,subject)):
#                     insert=os.path.join(path, subject, scan)
#                     subdir=os.listdir(insert)
#                     for item in subdir:
#                         if item.startswith('0'):
#                             insert2=os.path.join(path, subject, scan, item)
#                             dicom1=os.listdir(insert2)[0]
#                             insert3=os.path.join(path, subject, scan, item, dicom1)
#                             save_me=output.append(insert3)  
#     return

# dicom2nii_get_path('%s' % path)